In [ ]:
import argparse
import requests
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

API = "https://api.blockchain.info/charts/hash-rate"

UNIT_MULT = {
    "H/s": 1,
    "KH/s": 1e3,
    "MH/s": 1e6,
    "GH/s": 1e9,
    "TH/s": 1e12,
    "PH/s": 1e15,
    "EH/s": 1e18,
}

def fetch(timespan=None, start=None, end=None):
    params = {"format": "json"}
    if timespan:
        params["timespan"] = timespan
    if start:
        params["start"] = start
    if end:
        params["end"] = end
    r = requests.get(API, params=params, timeout=15)
    r.raise_for_status()
    return r.json()

def parse_values(js):
    unit = js.get("unit", "")
    mult = UNIT_MULT.get(unit, 1)  # fallback: assume H/s if unknown
    vals = js.get("values", [])
    xs = [datetime.datetime.fromtimestamp(v["x"]) for v in vals]
    ys = [float(v["y"]) * mult for v in vals]  # raw to H/s
    return xs, ys, unit

def plot(xs, ys, unit, out="hashrate.png", show=True):
    # Plot in EH/s for readability
    ys_eh = [y / 1e18 for y in ys]
    fig, ax = plt.subplots(figsize=(10,4))
    ax.plot(xs, ys_eh, linewidth=1)
    ax.set_ylabel("Hash rate (EH/s)")
    ax.set_title("Bitcoin Network Hash Rate")
    ax.grid(alpha=0.3)
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
    fig.tight_layout()
    fig.savefig(out, dpi=150)
    if show:
        plt.show()